# Synthèse Dask — Optimiser un pipeline et implémenter un Map/Reduce

## Objectifs pédagogiques
- Réviser les principes de base de Dask (`delayed`, calcul paresseux, `compute`, graphe de tâches).
- Identifier et corriger les erreurs classiques de parallélisation.
- Mettre en œuvre une logique **Map → Reduce** en Dask.
- Visualiser et interpréter le graphe de calcul distribué.

## Contexte

Un collègue a tenté de paralléliser le traitement d’un ensemble de fichiers contenant des nombres (un par ligne) pour calculer la **moyenne globale** de tous les fichiers.

Malheureusement, son script **ne tire pas parti de Dask** : il exécute les tâches séquentiellement et brise le graphe de calcul.

Vous devez :
1. Identifier les erreurs de conception ;
2. Réécrire le pipeline pour profiter du parallélisme de Dask ;
3. Étendre la solution avec une logique **Map/Reduce** claire.

## Code de départ (non optimal)

Voici le code initial fourni par votre collègue :

In [5]:
from dask import delayed

def read_file(filename):
    with open(filename, 'r') as f:
        numbers = [float(line.strip()) for line in f.readlines()]
    return numbers

def sum_len(numbers):
    return sum(numbers), len(numbers)

filenames = ["data/moyenne/1.txt", "data/moyenne/2.txt", "data/moyenne/3.txt"]

sum_len_vals = []
for f in filenames:
    numbers = delayed(read_file)(f).compute()
    sum_len_vals.append(sum_len(numbers))

global_mean = sum([s for s, n in sum_len_vals]) / sum([n for s, n in sum_len_vals])
print("Moyenne globale :", global_mean)

Moyenne globale : 4418314755.416667


**Question :** Créez des fichiers `"data/moyenne/1.txt"`, `"data/moyenne/2.txt"`, `"data/moyenne/3.txt"` contenant des nombres aléatoires pour tester le code ci-dessus.

## Analyse et correction du code

- Quel est le problème principal avec ce code ?
- Modifiez le dans le cadre ci-dessous pour qu'il utilise correctement Dask et parallélise les opérations.

In [6]:
from dask import delayed

def read_file(filename):
    with open(filename, 'r') as f:
        numbers = [float(line.strip()) for line in f.readlines()]
    return numbers

def sum_len(numbers):
    return sum(numbers), len(numbers)

filenames = ["data/moyenne/1.txt", "data/moyenne/2.txt", "data/moyenne/3.txt"]

sum_len_vals = []
for f in filenames:
    numbers = delayed(read_file)(f)
    sum_len_vals.append(delayed(sum_len)(numbers))

global_mean = sum(map(lambda t: t[0], sum_len_vals)) / sum(map(lambda t: t[1], sum_len_vals))

result = global_mean.compute()
print("Moyenne globale (pipeline) :", result)

Moyenne globale (pipeline) : 4418314755.416667


## Version Map/Reduce

On cherche maintenant à coder ce traitement dans le formalisme Map-Reduce.

**Question :** Comment découper le problème en étapes Map et Reduce ?

Voici une proposition de découpage :

- **Map** : lire chaque fichier et calculer une somme partielle et un compte.
- **Reduce** : combiner les résultats partiels pour obtenir la somme et le compte globaux.
- **Finalisation** : calculer la moyenne globale.

**Question :** Implémentez cette logique Map/Reduce en Dask dans le cadre ci-dessous.

In [ ]:
from dask import delayed
from functools import reduce
from collections import defaultdict

def partial_sum(filename):
    with open(filename, 'r') as f:
        numbers = [float(line.strip()) for line in f.readlines()]
    return (0, (sum(numbers), len(numbers)))

filenames = ["data/moyenne/1.txt", "data/moyenne/2.txt", "data/moyenne/3.txt"]

# Seules les MAP sont différées
mapped = [delayed(partial_sum)(f) for f in filenames]

# On force le compute ici : on récupère la liste de paires (key, (sum,count))
pairs = list(map(lambda t: t.compute(), mapped))

# Partitionnement et réduction exécutés immédiatement (séquentiel)
partitions = {}
for k, v in pairs:
    partitions[k] = partitions.get(k, []) + [v]

def combine(p1, p2):
    s1, n1 = p1
    s2, n2 = p2
    return s1 + s2, n1 + n2

results = {}
for k, vals in partitions.items():
    total = reduce(combine, vals)
    s, n = total
    results[k] = s / n

for k, mean in results.items():
    print("Clé:", k, "Moyenne (map-only delayed):", mean)

Clé: 0 Moyenne (map-only delayed): 4418314755.416667
